# Basic Animation Playback

This notebook will walk you through the basics of loading, inspecting, and playing back Choreography animations and sequences.

## Step 1: Connect to robot

In [ ]:
import logging
from spot_wrapper.wrapper import SpotWrapper

hostname="__ROBOT_IP__"
robot_name="__ROBOT_NAME__"
username="__USERNAME__"
password="__PASSWORD__"
port = 0
logger = logging.Logger(name="spot_wrapper_logger")

spot_wrapper = SpotWrapper(
    username=username, 
    password=password, 
    hostname=hostname, 
    port=port, 
    robot_name=robot_name, 
    logger=logger, 
    use_take_lease=True)

assert(spot_wrapper.is_valid)
spot_wrapper.claim()

## Uploading Animations and Sequences

Animations and sequences are stored in the robot's memory before playback. In the next cells we'll load an animation onto the robot and then prepare a sequence for playback that will trigger the animation.

In [ ]:
# Load Animation
from pathlib import Path
from spot_choreo_utils.paths import get_example_choreo_path
from spot_choreo_utils.serialization.serialization_utils import load_animation

animation = load_animation(Path(get_example_choreo_path(), "animations", "pose_to_pose_animation.pbtxt"))
spot_wrapper.upload_animation_proto(animation)

In [ ]:
# Load Sequence
from pathlib import Path
from spot_choreo_utils.paths import get_example_choreo_path
from spot_choreo_utils.serialization.serialization_utils import load_sequence

sequence = load_sequence(Path(get_example_choreo_path(), "sequences", "pose_to_pose_sequence.pbtxt"))

Let's take a look under the hood of animations and sequences.

In [ ]:
print(animation)

In [ ]:
print(sequence)

The animation starts with a name, and then hsa repeated blocks of data called animation_keyframes. Each of these keyframes specifies where the robot's body should be in space (relative to wherever the animation started) and joint angles for all the robot's joints. Each keyframe has a timestamp and during playback the robot will automatically interpolate between these joint positions as best it can.

The sequence is much shorter. The most important paramater is the animation_name, which matches the name in the animation definition. When you execute a dance on the robot and it encounters an animation_name, it will go check for that animation in the list of animations that have been uploaded. This is why all animations must be uploaded manually before playing back the choreography sequence.

Let's start the animation. Be sure to keep a safe distance from the robot before running the next cell.

In [ ]:
# Playback Animation
res, message = spot_wrapper.execute_dance(sequence, start_slice=0)
print(f"{res} - {message}")

Congratulations! You've played your first dance on Spot. Now let's start looking into how animations are built under the hood

# Create an animation from scratch. 

The next cell will build a simple animation to open and close the spot gripper by manually setting all of the properties through python instead of loading them in from a file.

In [ ]:
from bosdyn.api.spot.choreography_sequence_pb2 import (
    Animation,
    AnimationKeyframe,
    AnimateGripper
)
from google.protobuf.wrappers_pb2 import DoubleValue

GRIPPER_ANIMATION_NAME = "gripper_open_close"

# Define joint states
GRIPPER_OPEN_ANGLE = -1
GRIPPER_CLOSED_ANGLE = 0 

gripper_open = AnimateGripper()
gripper_open.gripper_angle.CopyFrom(DoubleValue(value=GRIPPER_OPEN_ANGLE))

gripper_close = AnimateGripper()
gripper_close.gripper_angle.CopyFrom(DoubleValue(value=GRIPPER_CLOSED_ANGLE))

# Create keyframe by combining joint states with timestamps
start_keyframe =  AnimationKeyframe()
start_keyframe.time = 0
start_keyframe.gripper.CopyFrom(gripper_close)

mid_keyframe =  AnimationKeyframe()
mid_keyframe.time = 1
mid_keyframe.gripper.CopyFrom(gripper_open)

end_keyframe =  AnimationKeyframe()
end_keyframe.time = 2
end_keyframe.gripper.CopyFrom(gripper_close)

# Add keyframes to an animation
gripper_animation_proto = Animation()
gripper_animation_proto.name = GRIPPER_ANIMATION_NAME

gripper_animation_proto.controls_gripper = True
gripper_animation_proto.animation_keyframes.append(start_keyframe)
gripper_animation_proto.animation_keyframes.append(mid_keyframe)
gripper_animation_proto.animation_keyframes.append(end_keyframe)

# Upload the animation
res, message = spot_wrapper.upload_animation_proto(gripper_animation_proto)
print(f"{res} - {message}")

That's an awful lot of code just to make a gripper open and close! And we haven't even set up the sequence to execute the animation. We'll set that up in the next cell.

In [ ]:
from bosdyn.api.spot.choreography_params_pb2 import (AnimateParams)
from bosdyn.api.spot.choreography_sequence_pb2 import (ChoreographySequence, 
                                                       MoveParams)

SEQUENCE_NAME = "programatic_sequence"
ANIMATION_NAME = GRIPPER_ANIMATION_NAME #"whole_body_animation"
# This paramater is more important within the Choreographer GUI so you can calculate how 
# BPMs translate into slices and set one keyframe for each 1/4 of a beat. For procedural animations
# we set the time directly so you might as well have an arbitrarily high precision for slices_per_minute
SLICES_PER_MINUTE = 6000
# Requested slices doesn't matter for a sequence that has only one move
# For multi-move sequences this should be claculated with slices per minute and animation length
REQUESTED_SLICES = 1

animation_params = AnimateParams()
animation_params.animation_name = ANIMATION_NAME

animation_move = MoveParams()
animation_move.type = "animation"
# Must be > 0
animation_move.start_slice = 1
animation_move.requested_slices = REQUESTED_SLICES
animation_move.animate_params.CopyFrom(animation_params)

template_sequence = ChoreographySequence()
template_sequence.name = SEQUENCE_NAME
template_sequence.slices_per_minute = SLICES_PER_MINUTE
template_sequence.moves.extend([animation_move])

Stand clear of the robot and run the next cell to see the gripper open and close

In [ ]:
res, message = spot_wrapper.execute_dance(template_sequence, start_slice=0)
print(f"{res} - {message}")

Congratulations! You've successfully loaded, built, and played choreography. But it took an awful lot of code to do it. In the next tutorial we'll look at some of the core functions within the spot_choreo_utils repo that can reduce the complexity of creating dances and speed up your process of creating exciting new motion with spot.

## Appendix
For more information on the choreography protobufs featured in this notebook, check out the official Boston Dynamics documentation.
https://dev.bostondynamics.com/protos/bosdyn/api/proto_reference#animation